# Kite
SIMcards with and without APN

In [103]:
import pandas as pd
import numpy as np
# Import datetime from the datetime module
from datetime import datetime

## Data

### Transform data from Kite

In [27]:
def getDataFromKite(csvFile):
    df = pd.read_csv(csvFile, sep=';', low_memory=False)
    df = df[['icc', 'imsi', 'msisdn',
            'apn_apn1',
            'apn_apn2',
            'apn_apn3',
            'apn_apn4',
            'apn_apn5',
            'apn_apn6',
            'apn_apn7',
            'apn_apn8',
            'apn_apn9',
            'apn_apn10']]
    df = df.applymap(str) # change all columns to string type
    df = df.applymap(lambda x: x.lstrip('=""').rstrip('"')) # remove unnecessary characteres
    
    return df

In [94]:
# !ls data

In [29]:
%%time
df = getDataFromKite('data/PAG02.csv')
df.shape

CPU times: user 43.6 s, sys: 2.69 s, total: 46.3 s
Wall time: 46.1 s


(1000000, 13)

In [96]:
list(df.columns)

['icc',
 'imsi',
 'msisdn',
 'apn_apn1',
 'apn_apn2',
 'apn_apn3',
 'apn_apn4',
 'apn_apn5',
 'apn_apn6',
 'apn_apn7',
 'apn_apn8',
 'apn_apn9',
 'apn_apn10']

In [95]:
# df.head()

### SIMs without any APN

In [64]:
df_no_apn = df[
    (df['apn_apn1'] == 'nan') &
    (df['apn_apn2'] == 'nan') &
    (df['apn_apn3'] == 'nan') &
    (df['apn_apn4'] == 'nan') &
    (df['apn_apn5'] == 'nan') &
    (df['apn_apn6'] == 'nan') &
    (df['apn_apn7'] == 'nan') &
    (df['apn_apn8'] == 'nan') &
    (df['apn_apn9'] == 'nan') &
    (df['apn_apn10'] == 'nan')
  ].copy()

df_no_apn.shape

(1171, 13)

### SIMs with at least one APN

https://stackoverflow.com/questions/44706485/how-to-remove-rows-in-a-pandas-dataframe-if-the-same-row-exists-in-another-dataf

In [92]:
%%time
df_apn = pd.merge(df, df_no_apn, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
df_apn.shape

CPU times: user 4.1 s, sys: 239 ms, total: 4.34 s
Wall time: 3.77 s


(998829, 13)

In [98]:
# validation
df_apn.shape[0] == df.shape[0] - df_no_apn.shape[0]

True

## Output

### Ooutput file

In [99]:
def getCurrentDate():
    local_dt = datetime.now()
    return str(local_dt.year) +'.'+ str(local_dt.month) + '.' + str(local_dt.day) + '_' + str(local_dt.hour) + 'h' + str(local_dt.minute)

In [107]:
# !mkdir data/result

In [108]:
df_no_apn.to_csv('data/result/pagseguro_no_apn' + getCurrentDate() + '.csv')

In [109]:
df_apn.to_csv('data/result/pagseguro_with_apn' + getCurrentDate() + '.csv')

In [110]:
!ls data/result

pagseguro_no_apn2021.6.8_15h23.csv  pagseguro_with_apn2021.6.8_15h23.csv
